In [ ]:
import numpy as np
import cv2
from skimage import io
import matplotlib.pyplot as plt

# For saving images
from google.colab import files

# Randomizer Function
def randomize_colour(colour):
    return np.random.randint(0, 256, size=3)

# ===== RELEVANT VARIABLES =====
# Boolean for using the randomizer functionality
# Currently it does not work IF num_colours is not 3
use_random = False
# Tolerance value for range checking (if two pixels have colours within this tolerance range of one another, they will be set to the same colour)
tolerance = 30
# Define the section size (in pixels)
section_size = 100 #320
# Randomization threshold (approximate percentage of sections with randomized colours)
threshold = 0.2
# Number of colours to use per section in the recolouring process
num_colours = 2
# ==============================

# Load the image -- provide the path
pxl_img = cv2.imread("PATH TO YOUR FILE")
# Recolours the image to rgb format for display purposes
pxl_img = cv2.cvtColor(pxl_img, cv2.COLOR_BGR2RGB)

# Get the image shape
height, width, _ = pxl_img.shape

# Loop through sections
for y in range(0, height, section_size):
  for x in range(0, width, section_size):
    # Extract the current section
    section = pxl_img[y:y+section_size, x:x+section_size, :]

    # Perform k-means clustering on the current section
    pixels = np.float32(section.reshape(-1, 3))
    # Controls the number of dominant colours per section
    n_colors = num_colours
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
    flags = cv2.KMEANS_RANDOM_CENTERS
    _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)

    if use_random:
      if np.random.rand() < threshold:
        # Randomize the colour to replace one of the dominant colors
        replacement_colour = randomize_colour(palette[np.random.choice(n_colors)])
        # If there is only one colour, the three palette values will be similar (not always the same for some reason)
        # so to be safe, we check if they are within a certain tolerance of one another

        # AT THE MOMENT THIS ONLY WORKS FOR 3 COLOURS
        # SELECTING "use_random = True" with any amount of colours other than "3" will cause a crash

        col_1 = palette[0]
        col_2 = palette[1]
        col_3 = palette[2]
        # Replace the colour
        if np.all(np.abs(col_1 - col_2) <= tolerance) and np.all(np.abs(col_2 - col_3) <= tolerance) and np.all(np.abs(col_1 - col_3) <= tolerance):
          palette[0] = replacement_colour
          palette[1] = replacement_colour
          palette[2] = replacement_colour
        else:
          palette[np.random.choice(len(palette))] = replacement_colour

    # Recolor the current section with the dominant colors
    section_recolored = np.uint8(palette[labels.flatten()]).reshape(section.shape)
    # Replace the original image with the recolored section
    pxl_img[y:y+section_size, x:x+section_size, :] = section_recolored

# Display the result
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(pxl_img)
ax.axis('off')
plt.show()

# Reformats the image back to bgr for proper colour encoding
pxl_img = cv2.cvtColor(pxl_img, cv2.COLOR_BGR2RGB)

In [9]:
# Download the image
cv2.imwrite("pixel 3.jpg", pxl_img)
files.download('pixel 3.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>